## Cross-filtering in dash

### Notebook preparation

Let's start by installing and importing the necessary libraries

In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [3]:
import dash
import json
import pandas as pd
import numpy as np
import plotly.express as px
import threading
from dash import dcc, html, Input, Output
from google.colab import output, drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


### Creating the visualization

#### Supporting solutions

Preparing the needed data

In [5]:
population = pd.read_csv('/content/drive/My Drive/population.csv')
new_books = pd.read_csv("/content/drive/My Drive/new-books-per-million.csv")
new_books = new_books[(new_books["Year"]>=1940) & (new_books["Year"]<=1996)]
selected_books = new_books[new_books["Year"]==1996][["Entity", "Book titles per capita (Fink-Jensen 2015)"]]
selected_pops = population[population["Year"]==1996][["Entity", "Population (historical estimates)"]]
data = pd.merge(selected_books, selected_pops, how="inner", on="Entity")

Creacting the chart used in the application

In [6]:
scatter_figure = px.scatter(data_frame=data,
                 x="Book titles per capita (Fink-Jensen 2015)",
                 y="Population (historical estimates)",
                 text="Entity",
                 log_x=True,
                 log_y=True,
                 template="none",
                 title="Country population vs number of new book titles per capita in 1996")
scatter_figure.update_traces(textposition="bottom right")

Below, we have the code of the first application used in the **dash crossfiltering** presentation.

In [7]:
app = dash.Dash()
app.layout = html.Div(
        children=[
                  html.P(id='div_value'),
                  dcc.Graph(id='scatter', figure=scatter_figure),
                  ],
)

@app.callback(
    Output('div_value', 'children'),
    Input('scatter', 'hoverData')
)
def show_me(data):
   return json.dumps(data)

Below, we have the code of the second application used in the **dash crossfiltering** presentation.

In [8]:
app = dash.Dash()
app.layout = html.Div(
    children=[
              dcc.Graph(
                  id='scatter',
                  figure=scatter_figure
                  ),
              dcc.Graph(id='line')
              ],
)

@app.callback(
    Output('line', 'figure'),
    Input('scatter','hoverData')
)
def build_line_chart(country):

  selected_country = country["points"][0]["text"]
  selected_data = new_books[new_books["Entity"]==selected_country]

  fig = px.line(data_frame=selected_data,
                x="Year",
                y="Book titles per capita (Fink-Jensen 2015)",
                template="none",
                title="Book titles per capita in %s for years 1940-1996" % selected_country
                )
  return fig

### Displaying the visualization

To build the visualization using the code above, we also need to run a server and go to the application page.


In [10]:
thread = threading.Thread(target=app.run)
thread.start()
output.serve_kernel_port_as_iframe(port=8050)

<IPython.core.display.Javascript object>